In [ ]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException,ElementNotInteractableException
from selenium.common.exceptions import WebDriverException
from selenium import webdriver
import time
import pandas as pd
import time

unclean_df = pd.DataFrame()
path = 'C:\\Users\\Dell\\OneDrive\\DataR Labs\\Resume_Parser\\chromedriver'
titles = pd.read_excel('C:\\Users\\Dell\\OneDrive\\DataR Labs\\Resume_Parser\\titles.xlsx')
#titles = titles[245:276]
titles = titles[397:]

no_of_jobs = 50
verb = False
sleept = 9

In [ ]:
def get_jobs(keyword, num_jobs,path,slp_time):
    
    #timer
    startTime = time.time()
    
    #driver
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)
    
    #counter and flags
    jobs = []
    num = 0
    i=1
    
    #url
    url ='https://www.monster.com/jobs/search?q='+ keyword +'&where=&geo=%27%27'
    jobs = []
    
    try:
        driver.get(url)
    except WebDriverException:
        print('Page Down')
        return pd.DataFrame(jobs)
    
    print("Extracting: "+keyword)
    
    #wait
    time.sleep(slp_time)
    
    #get parent id's
    listButtonsCount = driver.find_elements_by_class_name('results-card')
    job_buttons = driver.find_elements_by_xpath('//*[@class="title-company-location"]/a')
    
    #print(job_buttons.text)
    
    #links = driver.find_elements_by_class_name('title-company-location')
    #for link in links:
     #   print(link.text)
    for job_button in job_buttons:
        
        if not(num<5):
            #print(num)
            #print(num_jobs)
            executionTime = (time.time() - startTime)
            print('Execution time in seconds: ' + str(executionTime))
            return pd.DataFrame(jobs)
            
        #print(job_button.text)
        try:
            job_button.click()
        except (StaleElementReferenceException,ElementNotInteractableException) as e:
            pass
        
        #time.sleep(3)
        #print(driver.find_element_by_xpath('//div[contains(@class, "results-card selected")]/a'))
        
        #company name
        try:
            company_name = driver.find_element_by_xpath('//*[@id="app"]/div[2]/div/div[2]/div[2]/div[2]/div/div/div/div[1]/div/div[2]/div[1]').text
        except (NoSuchElementException,StaleElementReferenceException) as e:
            company_name = '-1'
        
        #location
        try:
            location = driver.find_element_by_xpath('//*[@id="app"]/div[2]/div/div[2]/div[2]/div[2]/div/div/div/div[1]/div/div[2]/div[2]/div[1]').text
        except (NoSuchElementException,StaleElementReferenceException) as e:
            location = '-1'
        
        #job title
        try:
            job_title = driver.find_element_by_xpath('//*[@id="app"]/div[2]/div/div[2]/div[2]/div[2]/div/div/div/div[1]/div/div[2]/h1').text
        except (NoSuchElementException,StaleElementReferenceException) as e:
            job_title = '-1'
        
        #job description
        try:
            job_description = driver.find_element_by_xpath("//div[contains(@class, 'jobdescriptioncomponent__SanitizedHtmlContainer-my61fv-2 gvwQGf')]").text
        except (NoSuchElementException,StaleElementReferenceException) as e:
            job_description = '-1'
        
        #job function
        try:
            job_function = driver.find_element_by_xpath('//*[@id="app"]/div[2]/div/div[2]/div[2]/div[2]/div/div/div/div[1]/div/div[2]/h1').text
        except (NoSuchElementException,StaleElementReferenceException) as e:
            job_function = '-1'
            
        try:
            j = str(i)
            link = driver.find_element_by_xpath('//*[@id="card-view-panel"]/div/div[1]/div['+ j +']/div[1]/div[1]/div/div[1]/div[2]/a').get_attribute('href')
            #print(driver.find_element_by_class_name("title-company-location").get_attribute("href"))
            #print(driver.find_element_by_xpath('//div[contains(@class, "results-card selected")]/a').get_attribute('href'))
            #print(driver.find_element_by_xpath("//div[@class='title-company-location']/a").get_attribute("href"))
            i = i+1
        except (NoSuchElementException,StaleElementReferenceException) as e:
            link = '-1'
        
        num = num+1
        jobs.append({"Job Title" : job_title,
                     "Searched Job Title" : keyword,
                     "Company Name" : company_name,
                     "Full Description" : job_description,
                     "Location" : location,
                     "URL" : link})
        #print("----- {0} {1} -----".format(num,keyword))

In [ ]:
import math

titles['NJ'] = ''
for rowIndex, row in titles.iterrows(): #iterate over rows
    k=0
    for columnIndex, value in row.items():
        value = str(value)
        if not (value=='NaN' or value == 'nan'):
            k=k+1
    titles['NJ'][rowIndex] = math.floor(40/k)

In [ ]:
df= pd.DataFrame()

In [ ]:
for rowIndex, row in titles.iterrows(): #iterate over rows
    k=0
    for columnIndex, value in row.items():
        value = str(value)
        if not (value=='NaN' or value == 'nan'):
            #print(titles['NJ'][rowIndex])
            df1 = get_jobs(value, titles['NJ'][rowIndex],path,sleept) 
        df = df.append(df1) 

# Clean

In [ ]:
import wordninja

import re
import unicodedata
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.util import ngrams


# Function to generate n-grams from sentences.
def extract_ngrams(data):
    l = []
    for i in range(1,6):
        n_grams = ngrams(nltk.word_tokenize(data), i)
        l.append( [' '.join(grams) for grams in n_grams])
    flat_ls = list(itertools.chain(*l))
    return (flat_ls)


def remove_accented_chars(x):
    """The function changes the accented characters into their equivalent normal form,
    to do so, normalize function with 'NFKD' is used which replaces the compatibility characters into
    theri euivalent
    
  param x(str): the sentence in which accented characters are to be detected and removes
  return x(str): sentence with accented characters replaced by their equivalent"""
    
    x = unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return x


def preprocess(df,d):
    """Preprocesses the given document by applying the following functionalities
  lower: lowers all the characters for uniformity
  remove special characters: using regex, removes all the punctuations etc
  remove space: removes trailing spaces and extra spaces between words
  remove accented characters: change accented characters to its normal equivalent
  remove stop words"""
  
    df[d] = df[d].apply(lambda x: x.lower())
    df[d] = df[d].apply(lambda x: re.sub('[^A-Z a-z 0-9-]+', '', x))
    df[d] = df[d].apply(lambda x: " ".join(x.split()))
    df[d] = df[d].apply(lambda x: remove_accented_chars(x))
    df[d] = df[d].apply(lambda x: " ".join([t for t in x.split() if t not in STOP_WORDS]))


def salary_extract(x):
    #change to string
    if x == -1:
        return 'Missing Value'
    return x.astype(str)    
    
def get_experience(x):
    """
    param
     -x : Textfield from where experience is to be extracted
    return
     -item : Returns year of experience if available, or returns -1 if not available 
    """
    
    exp = re.findall('\\d+\\s+(?:months?|years?)',x)
    if exp:
        for item in exp:
            return item
    else:
        return ('-1')    

def get_language(x):

    """
    Matched field with with language dictionary and returns languages if present in feild
    
    param
     -x : Textfield from where language is to be extracted
    return
     -item : Returns language if available, or returns -1 if not available 
    """
    
    language_dictionary = ['english',
                           'french',
                           'spanish',
                           'chinese',
                           'german',
                           'mandarin',
                           'japanese',
                           'russian']
    lang = ''
    
    for word in language_dictionary:
        if word in x:
            lang = lang + "|" + word
    if lang == '':
        return ('-1')
    else:
        return lang


def split_(k):
    l = wordninja.split(k)
    x = ''
    for word in l:
        x = x+word+' '
    return (x)

def get_responsibility(x):
    res = ['responsibilities',
           'responsibility',
           'function',
           'activities',
           'activity',
           'responsible',
           'role',
           'roles',
           'day',
           'workday',
           'day to day']
    flag = False
    #x = x.split()
    #print(x)
    
    for word in res:
        #print(word)
        try:
            pos = x.index(word)
            flag = True
            return (x[pos+len(word):pos+300])
        except ValueError:
            flag = False
    return ('No Value')

def get_skills(x):
    res = ['sucessful candidates',
           'skills',
           'expectation',
           'ideal candidate will have',
           'ideal candidate will possess',
           'skills you have',
           'you possess',
           'skill',
           'requirement',
           'you are an ideal candidate if',
           'skills required']
    flag = False
    #x = x.split()
    #print(x)
    
    for word in res:
        #print(word)
        try:
            pos = x.index(word)
            flag = True
            return (x[pos+len(word):pos+300])
        except ValueError:
            flag = False
    return ('No Value')


import re

certis = {}
for i in df['Job Title']:
    certis[i] = ""

cert = pd.read_csv('C:\\Users\\Dell\\OneDrive\\DataR Labs\\Resume_Parser\\certificates.csv') 
for i in range(0,len(cert)):
    certis[cert['Job'][i].rstrip().lstrip()] = cert['Certificates'][i]


certifications = []
for ind in range(0,len(df)):
    
    for cer in certis[df['Job Title'][ind]].split(","):
        certitext = ''
        
        if(re.search(cer.lstrip().rstrip(),df['Full Description'][ind])):
            certitext = cer+ " "+certitext 
        
    if(certitext==''):
            certitext= 'No Value'
        
    certifications.append(certitext)
    
    
educ = []
educate = pd.read_csv('EducationWords.csv')
#educate = pd.read_csv('/content/drive/MyDrive/data/EducationWords.csv') 
for i in educate['Education']:
    educ.append(i)

def get_education(x):
    educa =""
    education = []
    for ed in educ:
        if(re.search(ed.lower(),x.lower())):
            educa = ed+" "+educa
            education.append(educa)
            
    if len(education)!=0:
        return (education[-1])
    else:
        return 'No Value'

In [ ]:
df['Full Description'] = df['Full Description'].astype(pd.StringDtype())
preprocess(df,'Full Description')
df['Full Description'] = df['Full Description'].apply(split_)
df['Experience in years'] = df['Full Description'].apply(get_experience)
df['Experience in years'] = df['Experience in years'].map(lambda x : x.split(' ')[0])
df['bow'] = df['Full Description'].apply(extract_ngrams)
df['Foreign Language'] = df['Full Description'].apply(get_language)
df['Responsibilities'] = df['Full Description'].apply(get_responsibility)
df['Skills'] = df['Full Description'].apply(get_skills)
df['Certification'] = certifications
df = df.replace(' ','No Value')
df['Education'] = df['Full Description'].apply(get_education)